<a href="https://colab.research.google.com/github/alxmzr/Colab/blob/main/Supercharge_Technical_Analysis_with_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Getting Historical Bulk Data for Download
#https://medium.com/datadriveninvestor/supercharge-technical-analysis-with-machine-learning-af200909d824

def get_hist_data (tickers=None,time_window =2000,start_date=None,delta =1,end_date=None):

## Getting Historical Data
data = get_hist_data(tickers=top_100,time_window=2000,delta=10)

#Writing Data
data.to_csv(r"G:\My Drive\Personal Projects\Project Curator\Oct 2022\Data_Top100_SP500.csv")

# Importing Saved Data

data = pd.read_csv(r"G:\My Drive\Personal Projects\Project Curator\Data_Top100_SP500.csv",index_col=[0])


In [ ]:
# Example of labels AAPL
aapl = data[data["ticker"]=="AAPL"]
aapl_labelled = get_labels(data=aapl)
aapl_labelled.tail(10)

In [ ]:
##Getting Patterns

def get_patterns(data=None,ticker_column = 'ticker',mute=False):
    #from talib import abstract
    result = pd.DataFrame()
    tickers = data[ticker_column].unique()
    data = data.sort_index(ascending=True)
    x=[]
    attr = talib.get_function_groups()['Pattern Recognition']
    for i in tickers:
        df = data[data[ticker_column]==i]
        for a in attr:
            df[a] = (getattr(abstract,a)(df))/100
        result = pd.concat([result,df])
        x.append(i)
        if mute is False:
            print(i + " Patterns Appended Successfully")
            print(result.shape)
            print(str(len(x)) + " of "+ str(len(tickers)) + " Done ")
        else:
            pass
    return result

In [ ]:
#Creating Indicators

def get_indicators(data=None,ticker_column = 'ticker',price_column="close",mute=False):
    result = pd.DataFrame()
    tickers = data[ticker_column].unique()
    x=[]
    data = data.sort_index(ascending=True)
    for i in tickers:
        df = data[data[ticker_column]==i]
        df["Vol_over_30DAvg"] = df["volume"]/df["volume"].rolling(window=30).mean()
        df["Vol_over_10DAvg"] = df["volume"]/df["volume"].rolling(window=10).mean()
        df["Perc_From_200D_H"] = ((df[price_column]/df[price_column].rolling(window=200).max())-1)*100
        df["Perc_From_200D_L"] = ((df[price_column]/df[price_column].rolling(window=200).min())-1)*100
        df["Retracement_200D"] = df[price_column]/(df[price_column].rolling(window=200).max()-df[price_column].rolling(window=30).min())
        df["Perc_From_30D_H"] = ((df[price_column]/df[price_column].rolling(window=30).max())-1)*100
        df["Perc_From_30D_L"] = ((df[price_column]/df[price_column].rolling(window=30).min())-1)*100
        df["Retracement_30D"] = df[price_column]/(df[price_column].rolling(window=30).max()-df[price_column].rolling(window=30).min())
        df["30D/200D_High"] = df[price_column].rolling(window=30).max()/df[price_column].rolling(window=200).max()
        df["30D/200D_Low"] = df[price_column].rolling(window=30).min()/df[price_column].rolling(window=200).min()
        df["Perc_from 60DEMAVG"] = ((df[price_column]/df[price_column].rolling(window=60).mean())-1)*100
        df["Perc_from 30DEMAVG"] = ((df[price_column]/df[price_column].rolling(window=30).mean())-1)*100
        df["Perc_from 200DEMAVG"] = ((df[price_column]/df[price_column].rolling(window=200).mean())-1)*100
        df["Variand_30D/200D"] = df[price_column].rolling(window=30).var()/df[price_column].rolling(window=200).var()
        df = df.dropna()
        df = df.sort_index(ascending=True)
        result = pd.concat([result,df])

        if mute is False:
            print(i + " Indicators Appended Successfully")
            print(result.shape)
            x.append(i)
            print(str(len(x)) + " of "+ str(len(tickers)) + " Done ")
        else:
            pass

    return result

In [ ]:
## Getting Labels

def get_labels(data = None,ticker_column="ticker", fwd_window=1,price_column="close",threshold_perc=1,mute=False):
    result = pd.DataFrame()
    tickers = data[ticker_column].unique()
    x=[]
    data = data.sort_index(ascending=True)
    for i in tickers:
        df = data[data[ticker_column]==i]
        df.sort_index(ascending=False)
        df["Change"] = df[price_column].pct_change(periods=fwd_window).shift(-fwd_window)
        df["Change"]=np.where(abs(df["Change"])>=(threshold_perc/100),df["Change"]/abs(df["Change"]),0)
        result= pd.concat([result,df],axis=0)
        if mute is False:
            print(i + " Labels Appended Successfully")
            print(result.shape)
            x.append(i)
            print(str(len(x)) + " of "+ str(len(tickers)) + " Done ")
        else:
            pass
    result.dropna(inplace=True)
    return result

In [ ]:
## Compile Data

def compile_data(data=None,ticker_column = 'ticker',price_column="close",train=True,
fwd_window=1,threshold_perc=1,mute=False):
    print("Compiling pattern Signals")
    data_patterns = get_patterns(data=data,ticker_column=ticker_column,mute=mute)
    print ("---- Pattern Signals Attached")
    print("Compiling Indicators")
    data_indicators = get_indicators(data=data_patterns,ticker_column=ticker_column,
    price_column=price_column,mute=mute)
    print ("---- Indicators Attached")
    print ("Creating Labels")
    if train is True:
        data_labelled = get_labels(data=data_indicators,ticker_column=ticker_column,
        price_column="close", fwd_window=fwd_window,threshold_perc=1,mute=mute)
        print("---- All Labels Attached")
        return data_labelled
    else:
        return data_indicators

#Getting Pattern based signals

def get_pattern_signals(tickers=None,top=5):
    d=pd.DataFrame()
    start_date = date.today() - timedelta(days=300)
    start_date= start_date.strftime('%m/%d/%Y')
    end_date = date.today().strftime('%m/%d/%Y')
    for i in tickers:

        a = si.get_data(ticker = i, start_date=start_date,end_date=end_date)
        z = get_patterns(data=a,mute=True).iloc[-1,6:]

        d= pd.concat((d,z),axis=1)
        print(i + "   ",end="\r")
    d.columns = d.iloc[0]
    d=d.drop("ticker")
    d=d.sum()/d.count()
    d=d.sort_values(ascending=False)
    top_buy = d[d>0].sort_values(ascending=False).head(top)
    top_sell = d[d<0].sort_values(ascending=True).tail(top)
    print("Top Buy Ideas")
    print(top_buy)
    print("Top Sell Ideas")
    print(top_sell)
    return top_buy,top_sell

# Getting predictions

def get_signals (data=None,tickers=None,model=None,ticker_column = 'ticker',price_column="close",top=5):

    if data is None:
        data=get_hist_data(tickers=tickers,time_window=300,delta=0)

    else:
        data=data
        tickers=data[ticker_column].unique()

    result = pd.DataFrame()
    for i in tickers:
        compiled_data = compile_data(data=data[data[ticker_column]==i],ticker_column = 'ticker',
        price_column="close",train=False,mute=True)
        prediction = predict_model(model,data=compiled_data).iloc[-1:,-2:]
        result=pd.concat([result,prediction],axis=0)
        print(i + "- Prediction Attached")
    result["ticker"] = tickers
    top_buy = result[result["Label"]>0].sort_values(by=['Score'],ascending=False).head(top)
    top_sell = result[result["Label"]<0].sort_values(by=['Score'],ascending=False).head(top)

    return top_buy,top_sell

## Getting Top MCap Tickers

def get_top_tickers (index_name = 'sp500',top=50, tickers = None):
    ''' For index_name, you can choose from the following:\n
        'sp500'   - S&P 500 Index \n
        'dow'     - Dow Jones Industrial Average \n
        'nasdaq'  - Companies listed on Nasdaq\n
        'nifty50' - For NIFTY50\n
        'ftse100' - For FTSE100\n
        'ftse250' - For FTSE250\n
        Or for any other index not listed above, a list of all the tickers can be passed to
        filter out Top N companies by Market Capitalization '''
    mcap=[]
    if tickers is None:
        t = "tickers_"+index_name
        tickers = getattr(si,t)()
    else:
        pass
    for i in tickers:
        a= si.get_quote_data(i)
        b=round(a['sharesOutstanding']*a['regularMarketPrice']/1000000)
        mcap.append(b)
    mcap_df = pd.DataFrame()
    mcap_df["Tickers"] = tickers
    mcap_df["MCap_mn"] = mcap
    mcap_df=mcap_df.sort_values(by=["MCap_mn"],ascending=False)
    top_mcap_tickers = mcap_df.head(top)["Tickers"]
    return top_mcap_tickers.to_list()

## Getting Historical Bulk Data for Download

def get_hist_data (tickers=None,time_window =2000,start_date=None,delta =1,end_date=None):
    i_list=[]
    data = pd.DataFrame()

    if end_date is None:
        end_date = date.today() - timedelta(days=delta)
    else:
        end_date= end_date.strftime('%m/%d/%Y')

    if start_date is None:
        start_date = end_date - timedelta(days= time_window)
        start_date= start_date.strftime('%m/%d/%Y')
    else:
        start_date= start_date.strftime('%m/%d/%Y')

    for i in tickers:
        a = si.get_data(ticker = i, start_date=start_date,end_date=end_date)
        data = pd.concat([data,a],axis=0)
        print(i+" Appended Successfully")
        print(data.shape)
        i_list.append(i)
        print(str(len(i_list)) + " of "+ str(len(tickers)) + " Done ")
        data.drop_duplicates(inplace = True)
        data.dropna(inplace=True)
    return data